In [1]:
import requests

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [4]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
from elasticsearch import Elasticsearch

In [7]:
es_client = Elasticsearch('http://localhost:9200') 

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [11]:
query = 'How do execute a command on a Kubernetes pod?'

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

es_client.search(index=index_name, body=search_query)

ObjectApiResponse({'took': 234, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'eP4wSZcBk-79UN3Lbtv9', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'B_4wSZcBk-79UN3LcNzl', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kuber

In [17]:
query = "How do copy a file to a Docker container?"
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

context = ""

context_template = """
Q: {question}
A: {text}
""".strip()

for hit in response['hits']['hits']:
    context += f"{context_template.format(question=hit['_source']['question'], text=hit['_source']['text']).strip()}\n\n"

In [18]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

query = "How do I execute a command in a running docker container?"

prompt = prompt_template.format(question=query, context=context).strip()

In [19]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1/",
    api_key="ollama"
)

response = client.chat.completions.create(
    model= 'deepseek-r1:1.5b', # 'gemma3:1b',
    messages=[{"role": "user", "content": prompt}]
)
response.choices[0].message.content

"<think>\nOkay, I need to figure out how to execute a command in a running Docker container based on the context provided. Let me go through each part of the question and context to understand what's needed.\n\nFirst, the user is asking about executing commands inside a Docker container that's already running. From the context, I see there are two examples: `docker run -it` which starts a specific application and `docker exec -it <container-id>` for executing commands directly in the container.\n\nThe query is pretty straightforward but requires knowing how to get the container's entrypoint when it's not explicitly set. Oh, right, looking at the context, when using `docker run`, if you don't override the entrypoint, it defaults to 2. The user might need to use `-it` followed by an explicit entrypoint to start the container in a shell-like environment.\n\nThe assistant's answer from earlier explains that using `-it bash` makes it start like bash. Additionally, if it's already running, t

In [20]:
len(prompt)

1462

In [28]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

tid = encoding.encode(prompt)

In [32]:
len(tid)

322